In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network
from pyproj import Proj, transform

In [ ]:
# Creating the network for Brussels. 
# IMPORTANT: Use the same parameter values for the buffer as was done in STA_prep_script!
network = road_network_from_place("Brussels", buffer_dist_close=4000)
network = relabel_graph(network)
show_network(network,notebook=True)


In [ ]:
# Read in the zoning file
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'
zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + "BRUSSEL_3_10" + os.path.sep + "BRUSSEL_3_10.shp"
zoning = gpd.read_file(zoning_path) # Everything is now stored in a geopandas dataframe. 

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
connector_type = 'link'
network = add_centroids(network, x_centroids,y_centroids,k=1, method=connector_type)
network = relabel_graph(network)
show_network(network, notebook = true)

In [ ]:
# Read in the od matrix 
od_path = data_path + os.path.sep + "od_matrix_data_ext_tr" + os.path.sep + "BRUSSEL_ext_tr_3_9_10.xlsx"
od = pd.read_excel(od_path)
od_array = od.to_numpy() # The OD matrix is now stored in a numpy array

# Create OD graph and plot demand
od_graph = od_graph_from_matrix(od_array,x_centroids,y_centroids) 
# demand plot is useless since there are too many OD-pairs 

In [ ]:
# The network, zoning file and od matrix have been correctly loaded and linked by now. 
# This means that we can now run an assignment on the real demand! 
tolls = np.zeros(network.number_of_edges())
assignment = StaticAssignment(network,od_graph, tolls)
methods = ['dial_b']
for method in methods:
    result = assignment.run(method)
    show_network(network, flows = result.flows, notebook=True, show_nodes=False)
    print(f'{method=} ran successfully')
# all static assignments return a result object that follows the same structure, see
# below
print(result.__dict__.keys())

In [ ]:
veh_hours = result.link_costs * result.flows
objective = sum(veh_hours)
print(objective)